<a href="https://colab.research.google.com/github/mpjuarez/practica1_SRC/blob/main/practica1_sis_recomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


> **1. Instalar e importar librerias**



In [ ]:
!pip install pandas numpy scikit-surprise

**2. Instalación de las librerias**

In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split

2. Cargar el conjunto de datos

In [ ]:
!git clone https://github.com/mpjuarez/practica1_SRC.git

Cloning into 'practica1_SRC'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 964.69 KiB | 3.91 MiB/s, done.


**3 Cargar el dataset de calificaciones, películas y presentar**

In [ ]:
#Cargar el dataset de calificaciones y películas
calificaciones = pd.read_csv('practica1_SRC/ratings.csv')
peliculas = pd.read_csv('practica1_SRC/movies.csv')

In [ ]:
# Muestra las primeras 10 filas de calificaciones
print("Primeras 10 filas de calificaciones:")
display(calificaciones.head(10))

# Muestra las primeras 10 filas de películas
print("\nPrimeras 10 filas de películas:")
display(peliculas.head(10))

Primeras 10 filas de calificaciones:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100



Primeras 10 filas de películas:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
#Matriz Usuario-Película:

# In[25]:

!pip install scipy
from scipy.sparse import csr_matrix

# Crear la matriz usuario-película y llenar NaN con ceros
user_movie_matrix = calificaciones.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Convertir la matriz a dispersa para mejorar la eficiencia
user_movie_sparse_matrix = csr_matrix(user_movie_matrix)



In [ ]:
# Configurar el modelo KNN para similitudes de coseno
#'algorithm',puede tomar cualquiera de estos valores ['auto', 'ball_tree', 'kd_tree', 'brute'
#algoritmo 'brure', de fuerza bruta, calcula la distancia al cuadrado de cada vector de características.

knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
knn.fit(user_movie_sparse_matrix)


# # Predicción y Recomendación
# Definimos una función que, para un usuario específico, encuentra usuarios similares y calcula una calificación para cada película no vista por el usuario, basada en las calificaciones de sus vecinos más cercanos.

# In[27]:


# Función de recomendación de películas
def recomendar_peliculas(user_id, num_recommendations=5):
    user_index = user_id - 1  # Ajustar para índice de la matriz

    # Obtener vecinos cercanos al usuario
    distances, indices = knn.kneighbors(user_movie_sparse_matrix[user_index], n_neighbors=10)

    # Obtener calificaciones del usuario y películas no vistas
    user_ratings = user_movie_matrix.iloc[user_index]
    peliculas_no_vistas = user_ratings[user_ratings == 0].index

    # Si el usuario ha visto todas las películas, retornar sin recomendaciones
    if peliculas_no_vistas.empty:
        print(f"El usuario {user_id} ya ha visto todas las películas.")
        return pd.DataFrame(columns=['title'])

    # Almacenar los puntajes ponderados de recomendaciones
    recomendacion_scores = {}
    for pelicula in peliculas_no_vistas:
        score, suma_similitud = 0, 0
        for i in range(1, len(distances.flatten())):
            vecino_index = indices.flatten()[i]
            vecino_similitud = 1 - distances.flatten()[i]
            vecino_calificacion = user_movie_matrix.iloc[vecino_index][pelicula]
            if vecino_calificacion > 0:
                score += vecino_similitud * vecino_calificacion
                suma_similitud += vecino_similitud

        # Asignar puntaje promedio ponderado si hay similitud suficiente
        if suma_similitud > 0:
            recomendacion_scores[pelicula] = score / suma_similitud

    # Validar que existan recomendaciones
    if not recomendacion_scores:
        print("No se encontraron recomendaciones suficientes.")
        return pd.DataFrame(columns=['title'])

    # Ordenar y seleccionar las recomendaciones más altas
    peliculas_recomendadas = sorted(recomendacion_scores.items(), key=lambda x: x[1], reverse=True)
    recomendaciones_ids = [id for id, _ in peliculas_recomendadas[:num_recommendations]]
    # Reemplazar 'movies' con 'peliculas'
    recomendaciones = peliculas.set_index('movieId').loc[recomendaciones_ids][['title']]

    return recomendaciones


# # Presentar Recomendaciones para un Usuario específico:
# Se llama a la función de recomendación para un usuario específico, pasando el user_id y el número de recomendaciones que queremos obtener.

# In[23]:


#Ejemplo de recomendaciones para un usuario específico
user_id = 1  # Cambia el ID del usuario para probar con otros usuarios
print(f"Recomendaciones para el usuario {user_id}:")
print(recomendar_peliculas(user_id))

Recomendaciones para el usuario 1:
                                                     title
movieId                                                   
1243          Rosencrantz and Guildenstern Are Dead (1990)
3022                                   General, The (1926)
514                                        Ref, The (1994)
529                     Searching for Bobby Fischer (1993)
720      Wallace & Gromit: The Best of Aardman Animatio...


**texto en dddddddddddnegrita**